In [2]:
import pandas as pd
from src.static import DATA_DIR

In [19]:
mapped_data = pd.read_csv(f'{DATA_DIR}/mapped_data.csv', na_values=['.','NaN', 'None'])
mapped_data.sort_values(by=['year', 'enterprise_flag', 'record_number'], inplace=True)

In [28]:
mapped_data[mapped_data.record_number == 1]

,year,enterprise_flag,record_number,census_tract_2020,tract_income_ratio,affordability_cat,date_of_mortgage_note,purpose_of_loan,type_of_seller,federal_guarantee,tot_num_units,num_bedrooms,num_units,affordability_level,tenant_income_ind
315783,2008,fannie,1,<10%,">10, <=120%","<20%, >=40%",prior to year aquired,refinance,mortgage_company,no,120,>=2,1.0,">50, <=60%",No
315784,2008,fannie,1,<10%,">10, <=120%","<20%, >=40%",prior to year aquired,refinance,mortgage_company,no,120,0-1,10.0,">50, <=60%",No
315785,2008,fannie,1,<10%,">10, <=120%","<20%, >=40%",prior to year aquired,refinance,mortgage_company,no,120,0-1,1.0,">50, <=60%",No
315786,2008,fannie,1,<10%,">10, <=120%","<20%, >=40%",prior to year aquired,refinance,mortgage_company,no,120,>=2,4.0,">50, <=60%",No
315787,2008,fannie,1,<10%,">10, <=120%","<20%, >=40%",prior to year aquired,refinance,mortgage_company,no,120,0-1,3.0,">=0, <=50%",No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
608485,2023,freddie,1,>=30% <100%,">10, <=120%",">=20%, <40%",same year as acquired,refinance,mortgage_company,no,51-99,0-1,3.0,">60, <=80%",No
608486,2023,freddie,1,>=30% <100%,">10, <=120%",">=20%, <40%",same year as acquired,refinance,mortgage_company,no,51-99,0-1,2.0,">=0, <=50%",No
608487,2023,freddie,1,>=30% <100%,">10, <=120%",">=20%, <40%",same year as acquired,refinance,mortgage_company,no,51-99,0-1,6.0,">=0, <=50%",No
608488,2023,freddie,1,>=30% <100%,">10, <=120%",">=20%, <40%",same year as acquired,refinance,mortgage_company,no,51-99,0-1,3.0,">=0, <=50%",No


In [21]:
mapped_data.num_units.dtypes

dtype('float64')

In [22]:
df = pd.DataFrame(None)

In [ ]:
gb = mapped_data.groupby(['year', 'enterprise_flag', 'record_number', 'affordability_level', 'num_bedrooms'])
df['sum_num_units'] = gb['num_units'].sum()
df['pct_minority'] = 

In [ ]:
df

sum_num_units
year enterprise_flag record_number affordability_level num_bedrooms               
2008 fannie          1             >50, <=60%          0-1                    53.0
                                                       >=2                    29.0
                                   >=0, <=50%          0-1                     6.0
                                                       >=2                     1.0
                     2             >60, <=80%          0-1                    10.0
...                                                                            ...
2023 freddie         2596          >60, <=80%          >=2                     3.0
                                   >=0, <=50%          0-1                     6.0
                                                       >=2                    96.0
                     2597          >50, <=60%          >=2                    44.0
                                   >=0, <=50%          >=2                    35.0

[397624 rows x 1 columns]